Author: Yoseph Loyd
Date 20241212

Developement of a means to segment capilarries in tissue images. This is intended to be ported through the ChimeraX visualizer.

###initial commands for ChimaraX input test date###

open "C:/Users/LoydY/OneDrive - SUNY Upstate Medical University/Documents/20240917_006/006_decon_podocyte.tif";
volume #1 style surface step 2 level 1.449e+04;

volume #1 voxelSize 1,1,1.52;
marker #2 position 554.9,779.1,71.11 color yellow radius 1;
lighting depthcue false;

ui tool show shell

Python functions imports

In [ ]:
from chimerax.map_data import ArrayGridData 
from chimerax.map import volume_from_grid_data
import itertools
import numpy
import scipy
import skimage

Identification and rescaling of imagein ChimeraX ENV.

In [ ]:
y=session.models[2].volume.full_matrix().copy()
p= session.models[3].atoms.coords
p=numpy.round(p).astype('int')

matrix=numpy.zeros((numpy.shape(y)))
a=numpy.linspace(p[0,0]-80,p[0,0]+80, 161)
b=numpy.linspace(p[0,1]-80,p[0,1]+80, 161)
c=numpy.linspace((p[0,2]/1.5)-44,(p[0,2]/1.5)+27, 72)

q=numpy.array(list(itertools.product(*(a,b,c),repeat=1)),dtype='int')
w=numpy.split(q,3,1)
matrix[w[2],w[1],w[0]]=1
mask=ArrayGridData(matrix,step=(1,1,1.5))
volume_from_grid_data(mask,session)

Using the Bernsens auto thresholding approach I'll segement capillary walls in the ROI while using an auto global contrast threshold variable.

In [ ]:

welp=matrix*y
welp[welp==0] = numpy.nan

radius=3
help= numpy.zeros(numpy.shape(welp))
for h in range(numpy.min(w[2]),numpy.max(w[2])):
    for g in range(numpy.min(w[1]),numpy.max(w[1])):
        for f in range(numpy.min(w[0]),numpy.max(w[0])):
            fl=numpy.int32(numpy.linspace(f-radius,radius+f, 2*radius+1))-1
            gl=numpy.int32(numpy.linspace(g-radius,radius+g, 2*radius+1))-1
            hl=numpy.int32(numpy.linspace(h-radius,radius+h, 2*radius+1))-1
            if numpy.isnan((welp[hl,gl,fl].all()))==False:
                standardD = numpy.nanstd(welp[hl,gl,fl])*0.15
                Thresh= ((numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))/2)+standardD
                contrast= (numpy.nanmax(welp[hl,gl,fl])-numpy.nanmin(welp[hl,gl,fl]))
            else:
                help = numpy.nan
            
            if  welp[h,g,f] < contrast:
                welp[h,g,f]= 0
            else:
                welp[h,g,f]= welp[h,g,f]
            
            if welp[h,g,f] >= (Thresh):
                help[h,g,f]= 1
            else:
                help[h,g,f]=0